# Forced Convection over a flat plate

A flat plate of width w=1 m is maintained at a uniform surface temperature, 𝑇_𝑠= 230<sup>o</sup>C,
 by using independently controlled, electrical strip heaters, each of which is 50 mm long.  If
 atmospheric air at 25oC flows over the plate at a velocity of 60 m/s, at what heater is the
 electrical input a maximum? What is the value of this input?

# Given

In [1]:
import numpy as np

from thermal_system_calculation import FluidState, Fluid

width = 1 # m
length_heater = 0.05 # m
temp_surface = 230 # degC
temp_air = 25 # degC
velocity_air = 60 # m/s

# Find
At what heater is the electrical input a maximum?

What is the value of this maximum heat input?

# Schematic
![FlatSurfaceForceHeacConvection](images/flat_surface_convection_heating.png)

# Assumption
1. The temperature is uniform at the surface.
2. Steady state conditions
3. Uniform and constant properties
4. Uniform heat transfer over a single heater

# Properties

In [2]:
temp_boundary = np.mean([temp_air, temp_surface]) + 273.15
air_properties = FluidState(
    fluid=Fluid.AIR,
    temp_k=temp_boundary,
    characteristic_length=length_heater
)
thermal_conductivity_air = air_properties.k
specific_heat_capacity_air = air_properties.cp
dynamic_viscosity_air = air_properties.dynamic_viscosity
density_air = air_properties.density
print(f"Air at {temp_boundary} K has: \n"
      f"\t- thermal conductivity: {thermal_conductivity_air:.3f} W/mK\n"
      f"\t- dynamic viscosity: {dynamic_viscosity_air:.5e} Pa s \n"
      f"\t- density: {density_air:.3f} kg/m3")


Air at 400.65 K has: 
	- thermal conductivity: 0.033 W/mK
	- dynamic viscosity: 2.30830e-05 Pa s 
	- density: 0.881 kg/m3


# Analysis
As the temperature on the surface is give, we can calculate the heat transfer by the Newton's
cooling law.
$$q_\mathrm{conv} = h A \left(T_s - T_\infty\right)$$
and this heat transfer has to be balanced by the heat input from each heater.

We just learned that h is mainly determined by development of the boundary layers and the degree of
turbulence of the fluid flow. We need to search for maximum points, the entrance of the
 plate and near the flow transition from laminar flow to turbulence is. See below figure.
![h for forced convection along a flat plate](images/h_transition.png)

For the first heater, we can determine average heat transfer coefficients by convection and heat
transfer rate,
$\overline{h}_1$ by,
$$
\begin{align}
    \overline{Nu}_1 &= 0.664 Re_1^{1/2} Pr^{1/3} \\
    \overline{h}_1 &= \frac{\overline{Nu}_1 k}{L_1}
\end{align}
$$

In [3]:
# Let's define the functions for Reynold's number, Prandtl number and Nusselts number.
def get_reynolds_number(
        velocity: float,
        length: float,
        density: float,
        dynamic_viscosity
) -> float:
    return velocity * length * density / dynamic_viscosity

def get_prandtl_number(
        specific_heat_capacity: float,
        dynamic_viscosity: float,
        thermal_conductivity: float
) -> float:
    return specific_heat_capacity * dynamic_viscosity / thermal_conductivity

def get_h_from_nusselt_number(
        nusselt_number: float,
        thermal_conductivity: float,
        length: float
) -> float:
    return nusselt_number * thermal_conductivity / length

re1 = get_reynolds_number(velocity_air, length_heater, density_air, dynamic_viscosity_air)
pr = get_prandtl_number(specific_heat_capacity_air, dynamic_viscosity_air, thermal_conductivity_air)
nu1 = 0.664 * re1**0.5 * pr**(1/3)
h1 = get_h_from_nusselt_number(nu1, thermal_conductivity_air, length_heater)
q1 = h1 * width * length_heater * (temp_surface - temp_air)

print(f"The heat input from the first heater is {q1:.3f} W.")

The heat input from the first heater is 1369.114 W.


The next possible maximum heat input will happen when the air flow becomes fully turbulent. We
will use a criteria $Re_c = 500000$ for the full turbulence flow. We can find a characteristic
length for this $Re_c$.

In [4]:
reynolds_number_at_full_turbulence = 500000
length_full_turbulence = reynolds_number_at_full_turbulence * dynamic_viscosity_air / \
                         (density_air * velocity_air)
plate_number_full_turbulence = int(np.ceil(length_full_turbulence / length_heater))
print(f"The flow is fully turbulent after {length_full_turbulence:.3f} m, "
      f"at {plate_number_full_turbulence}th heater.")

The flow is fully turbulent after 0.218 m, at 5th heater.


On the 5th heater, the plate above has both laminar and turbulent flow. It has mixed boundary
layer conditions. We can obtain the local heat convection above the 5th heater by
$$
\begin{align}
    q_5 &= q_{1-5} - q_{1-4} \\
        &= \overline{h}_{1-5} L_5 w \left(T_s - T_\infty\right) - \overline{h}_{1-4}L_4 w \left(T_s - T_\infty\right) \\
        &= \left(\overline{h}_{1-5} L_5 - \overline{h}_{1-4} L_4\right) w \left(T_s - T_\infty\right)
\end{align}
$$
where we obtain $\overline{h}_{1-5}$ from the mixed boundary layer conditions and $\overline{h}_{1-4}$
from the laminar boundary layer condition.

$$
\begin{align}
    \overline{Nu}_{1-5} &= \left(0.037 Re_5^{4/5} - 871\right)Pr^{1/3} \\
    \overline{Nu}_{1-4} &= 0.664 Re_4^{1/2} * Pr^{1/3}
\end{align}
$$

In [5]:
# h_1_5 Average convection heat transfer coefficient from the beginning to the end of the 5th heater
char_length_5 = length_heater * 5
re5 = get_reynolds_number(
    velocity=velocity_air,
    length=char_length_5,
    density=density_air,
    dynamic_viscosity=dynamic_viscosity_air
)
# Pradntl number does not change with the flow condition.
nu_1_5 = (0.037 * re5**0.8 - 871) * pr**(1/3)
h_1_5 = get_h_from_nusselt_number(nu_1_5, thermal_conductivity_air, char_length_5)

# h_1_4 Average convection heat transfer coefficient from the beginning to the end of the 4th heater
char_length_4 = length_heater * 4
re4 = get_reynolds_number(
    velocity=velocity_air,
    length=char_length_4,
    density=density_air,
    dynamic_viscosity=dynamic_viscosity_air
)
nu_1_4 = 0.664 * re4**0.5 * pr**(1/3)
h_1_4 = get_h_from_nusselt_number(nu_1_4, thermal_conductivity_air, char_length_4)

q_5 = (h_1_5 * char_length_5 - h_1_4 * char_length_4) * width * (temp_surface - temp_air)

print(f"The heat input from the fifth heater is {q_5:.3f} W.")

The heat input from the fifth heater is 1058.754 W.


It is also possible that the heat transfer right after the transition to fully turbulent flow
since at the fifth section, it has a mixed boundary condition. We can see the heat transfer at
the sixth section in the same way as in the fifth section.h

$$q_6 = \left(\overline{h}_{1-6} L_6 - \overline{h}_{1-5} L_5\right) w \left(T_s - T_\infty\right)$$

In [6]:
char_length_6 = length_heater * 6
re6 = get_reynolds_number(
    velocity=velocity_air,
    length=char_length_6,
    density=density_air,
    dynamic_viscosity=dynamic_viscosity_air
)
nu_1_6 = (0.037 * re6**0.8 - 871) * pr**(1/3)
h_1_6 = get_h_from_nusselt_number(nu_1_6, thermal_conductivity_air, char_length_6)

q_6 = (h_1_6 * char_length_6 - h_1_5 * char_length_5) * width * (temp_surface - temp_air)

print(f"The heat input from the sixth heater is {q_6:.3f} W.")




The heat input from the sixth heater is 1429.741 W.


Therefore, the maximum heat input is required at the sixth heater.